In [ ]:
from numpy import array

from tensorflow.python.keras import backend as k
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, LSTM, Embedding

import itertools
 
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
        # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = array(encoded)
        # predict a word in the vocabulary
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [ ]:
def load_chinese(file):
    sentence_array = []
    with open(file) as infile:
        for line in infile:
            line = line.strip()
            current = []
            for i in line:
                if i != "-":
                    current.append(i)
            seg_str = " ".join(current)
            sentence_array.append(seg_str)
    return sentence_array

chinese_poi_name = load_chinese("./input_data/poi_1000000")

In [ ]:
chinese_poi_name

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(chinese_poi_name)
# tokenizer.word_index

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(["Jack and Jill went up the hill"])
# print(tokenizer.word_index)
# tokenizer.texts_to_sequences(["jack"])

In [ ]:
print(chinese_poi_name[0])
print(type(chinese_poi_name[0]))
print(tokenizer.word_index["铁"])

In [ ]:
tokenizer.texts_to_sequences([chinese_poi_name[0]])

In [ ]:
encoded = tokenizer.texts_to_sequences(chinese_poi_name)

In [ ]:
encoded[0:2]

In [ ]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# create word -> word sequences
def generate_word_pair(encoded):
    sequences = list()
    for i in encoded:
        tuple_temp = tuple(i)
        for pair in list(zip(i, i[1:])):
            sequences.append(pair)
    print('Total Sequences: %d' % len(sequences))
    return sequences

chinese_word_pair = generate_word_pair(encoded)


In [ ]:
print(type(chinese_word_pair))
print(chinese_word_pair[0:10])

In [ ]:
test = tuple([338, 15, 101, 112, 2, 24, 20])
print(type(test))
print(list(zip(test, test[1:])))

In [ ]:
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=0)

In [ ]:
# evaluate
print(generate_seq(model, tokenizer, 'Jack', 6))